<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [1]</a>'.</span>

# Mobile Get Input Notebook - Phase 4: Post-processing & Export

**Phase 4**: Format data and export to CSV for P.1812 batch processing.

This final phase takes the enriched GeoDataFrame from Phase 3, formats it for the ITU-R P.1812-6 model, and exports CSV profiles grouped by azimuth.

## Prerequisites
- Run **Phase 0-3** first (full pipeline)

## Workflow
1. **Load Phase 3 outputs**: Enriched GeoDataFrame
2. **Format for P.1812**: Extract distance arrays, height profiles, parameters
3. **Group by azimuth**: Create one profile per azimuth (36 profiles)
4. **Export CSV**: Save semicolon-delimited profiles for batch_processor.py

## Output
- CSV profiles in `data/input/profiles/`
- One profile per azimuth (0°, 10°, 20°, ... 350°)
- Semicolon-delimited, ready for P.1812 batch processing

## Setup: Import from Phases 0-3

Load all previous phase outputs.

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [1]:
# Import Phase 0 setup
%run phase0_setup.ipynb

# Import Phase 2 receiver points
%run phase2_batch_points.ipynb

# Import Phase 3 extraction (this loads Phase 0+2 again, but idempotent)
%run phase3_batch_extraction.ipynb

print("\n✓ All phases imported successfully")
print(f"  Enriched GeoDataFrame: {len(receivers_gdf)} points")

Exception: File `'phase0_setup.ipynb'` not found.

## Format Data for P.1812

Convert GeoDataFrame into profile format expected by P.1812 model.

In [ ]:
print("\n" + "="*60)
print("PHASE 4: POST-PROCESSING & EXPORT")
print("="*60)

# Get distance rings and azimuths
distance_rings = sorted(set([round(d) for d in receivers_gdf['distance_km'].dropna().unique() if d > 0]))
azimuths_found = sorted(receivers_gdf['azimuth_deg'].dropna().unique())
print(f"\nCreating {len(distance_rings)} x {len(azimuths_found)} = {len(distance_rings)*len(azimuths_found)} profiles...")

# Get parameters from CONFIG
frequency_ghz = CONFIG['P1812']['frequency_ghz']
time_percentage = CONFIG['P1812']['time_percentage']
polarization = CONFIG['P1812']['polarization']
htg = CONFIG['TRANSMITTER']['antenna_height_tx']
hrg = CONFIG['TRANSMITTER']['antenna_height_rx']

profiles = []

for ring_km in distance_rings:
    for az in azimuths_found:
        subset = receivers_gdf[
            (receivers_gdf['azimuth_deg'] == az) & 
            (receivers_gdf['distance_km'] <= ring_km + 0.05)
        ].sort_values('distance_km')
        if len(subset) == 0: continue
        distances = [0] + subset['distance_km'].tolist()
        heights = [int(round(h)) if h else 0 for h in subset['h'].tolist()]
        heights = [heights[0]] + heights
        r_vals = [r for r in subset['R'].tolist()]
        r_vals = [r_vals[0]] + r_vals
        ct_vals = [c for c in subset['Ct'].tolist()]
        ct_vals = [ct_vals[0]] + ct_vals
        zones = [z for z in subset['zone'].tolist()]
        zones = [zones[0]] + zones
        geom_0 = subset.geometry.iloc[0]
        geom_last = subset.geometry.iloc[-1]
        tx_lat, tx_lon = float(geom_0.y), float(geom_0.x)
        rx_lat, rx_lon = float(geom_last.y), float(geom_last.x)
        profiles.append({'f': frequency_ghz, 'p': time_percentage, 'd': distances,
            'h': heights, 'R': r_vals, 'Ct': ct_vals, 'zone': zones,
            'htg': htg, 'hrg': hrg, 'pol': polarization,
            'phi_t': tx_lat, 'phi_r': rx_lat,
            'lam_t': tx_lon, 'lam_r': rx_lon,
            'azimuth': az, 'distance_ring': ring_km})

print(f"✓ Formatted {len(profiles)} profiles")

## Export to CSV

Save profiles as semicolon-delimited CSV files.

In [ ]:
from datetime import datetime
import hashlib

print(f"\nExporting profiles to CSV (all distance rings + azimuths)...")

# Create DataFrame from all profiles
df_profiles = pd.DataFrame(profiles)

# Generate smart filename with metadata
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
tx_id = CONFIG['TRANSMITTER'].get('id', 'TX0')
num_azimuths = len(df_profiles['azimuth'].unique())
num_rings = len(df_profiles['distance_ring'].unique()) if 'distance_ring' in df_profiles.columns else 0
total_profiles = len(df_profiles)

# Create content hash for version tracking
csv_content = df_profiles.to_csv(sep=";", index=False, decimal=".")
content_hash = hashlib.md5(csv_content.encode()).hexdigest()[:8]

# Format: profiles_{tx_id}_{num_profiles}p_{num_azimuths}az_{max_dist}km_v{timestamp}_{hash}.csv
output_filename = f"profiles_{tx_id}_{total_profiles}p_{num_azimuths}az_{max_distance_km}km_v{timestamp}_{content_hash}.csv"
output_path = profiles_dir / output_filename

df_profiles.to_csv(
    output_path,
    sep=";",
    index=False,
    decimal="."
)

print(f"✓ Saved {len(df_profiles)} profiles to {output_path}")
print(f"\n📊 Profile Metadata:")
print(f"  Total profiles: {total_profiles}")
print(f"  File size: {output_path.stat().st_size / 1024:.1f} KB")
print(f"  Azimuths: {num_azimuths}")
print(f"  Distance rings: {num_rings}")
print(f"  Max distance: {max_distance_km} km")
print(f"  Transmitter ID: {tx_id}")
print(f"  Generated: {timestamp}")
print(f"  Content hash: {content_hash}")
print(f"\n📝 Filename format: profiles_{{TX_ID}}_{{PROFILES}}p_{{AZIMUTHS}}az_{{DISTANCE}}km_v{{TIMESTAMP}}_{{HASH}}.csv")

## Validation & Comparison

Verify output and compare against original workflow.

In [ ]:
print(f"\nCSV Validation:")
print(f"  Profiles: {len(df_profiles)}")
print(f"  Azimuths: {sorted(df_profiles['azimuth'].unique())}")
print(f"  Distance range: {min([len(d) for d in df_profiles['d']])} - {max([len(d) for d in df_profiles['d']])} points per profile")

# Sample profile details
sample = df_profiles.iloc[0]
print(f"\nSample profile details (azimuth {sample['azimuth']}°):")
print(f"  TX: ({sample['phi_t']:.4f}, {sample['lam_t']:.4f})")
print(f"  RX: ({sample['phi_r']:.4f}, {sample['lam_r']:.4f})")
print(f"  Frequency: {sample['f']} GHz")
print(f"  Time %: {sample['p']}%")
print(f"  Antenna heights: TX={sample['htg']}m, RX={sample['hrg']}m")
print(f"  Distance points: {len(sample['d'])}")
print(f"  Height range: {min(sample['h'])}-{max(sample['h'])}m")
print(f"  Ct classes: {set(sample['Ct'])}")

print(f"\n" + "="*60)
print("PHASE 4 COMPLETE: Export ready for batch processing")
print("="*60)
print(f"\nOutput file: {output_path}")
print(f"Ready to run: python scripts/run_batch_processor.py")

## Summary

**Phase 4 Complete**:
- ✓ Data formatted for P.1812 model
- ✓ Grouped by azimuth (36 profiles)
- ✓ Exported to semicolon-delimited CSV
- ✓ Ready for batch_processor.py

**Full Pipeline Complete**:
- Phase 0: Setup ✓
- Phase 1: Land cover ✓
- Phase 2: Batch points (~0.1s) ✓
- Phase 3: Batch extraction (~15-20s with Optimization A) ✓
- Phase 4: Export ✓
- **Total**: ~20-25s (vs ~170s without optimization)

**Next**: Use batch_processor.py to process profiles through P.1812-6 model